In [ ]:
!experiment_name="dcnet";cd ../ ;\
    name_date="${experiment_name}/$(date +%Y-%m%d-%H%M%S)";\
    dir="./results/${name_date}";mkdir -p ${dir};dir_log="${dir}/output.log";\
CUDA_VISIBLE_DEVICES=2,3 torchrun --nproc_per_node=2 train_multi_GPU.py \
    --wandb True --wandb_model dryrun --sync_bn False --amp True --aux False \
    --batch_size 5 --batch_size_val 4 --memory_size 32768 \
    --epochs 100 \
    --data_path cityscapes/Cityscape --num_classes 18 \
    --cfg datasets/dcnet.yaml \
    --model_name dcnet_resnet50 --pre_trained pre-trained/pytorch/fcn_resnet50_coco.pth \
    --contrast 0 --loss_name double --L1_loss 0.1 --L2_loss 0.1 --L3_loss 0.1\
    --name_date $name_date \
    2>&1 | tee $dir_log